<a href="https://colab.research.google.com/github/anzhelikasuchkova/Neural_Network_Charity_Analysis/blob/main/AlphabetSoupCharity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Deliverable 1: Preprocessing the Data for a Neural Network

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv
url = 'https://raw.githubusercontent.com/anzhelikasuchkova/Neural_Network_Charity_Analysis/main/charity_data.csv'

application_df = pd.read_csv(url)
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns="EIN")
application_df  = application_df.drop(columns="NAME")
application_df

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_type= application_df["APPLICATION_TYPE"].value_counts()
application_type

In [ ]:
# Visualize the value counts of APPLICATION_TYPE
application_df["APPLICATION_TYPE"].value_counts().plot.density()

In [ ]:
# Determine which values to replace if counts are less than ...?
replace_application= application_type[application_type < 500].index.tolist()

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")
    
# Check to make sure binning was successful
application_df.APPLICATION_TYPE.value_counts()

In [ ]:
# Look at CLASSIFICATION value counts for binning
classification_value=application_df["CLASSIFICATION"].value_counts()
classification_value.head(50)

In [ ]:
# Visualize the value counts of CLASSIFICATION
classification_value.plot.density()

In [ ]:
# Determine which values to replace if counts are less than ..?
replace_class = list(classification_value[classification_value < 500].index)

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")
    
# Check to make sure binning was successful
application_df.CLASSIFICATION.value_counts()

In [ ]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_cat

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(application_cat)
encode_df.head()

In [ ]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True,right_index=True)
application_df = application_df.drop(columns = application_cat)
application_df.head()

In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(['IS_SUCCESSFUL'],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### Deliverable 2: Compile, Train and Evaluate the Model

In [116]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_36 (Dense)            (None, 80)                3600      
                                                                 
 dense_37 (Dense)            (None, 30)                2430      
                                                                 
 dense_38 (Dense)            (None, 1)                 31        
                                                                 
Total params: 6,061
Trainable params: 6,061
Non-trainable params: 0
_________________________________________________________________


In [117]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [118]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,validation_split=0.15, epochs=100)

Epoch 1/100
684/684 [==============================] - 3s 3ms/step - loss: 0.5735 - accuracy: 0.7170 - val_loss: 0.5503 - val_accuracy: 0.7313
Epoch 2/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5575 - accuracy: 0.7269 - val_loss: 0.5471 - val_accuracy: 0.7383
Epoch 3/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5534 - accuracy: 0.7272 - val_loss: 0.5456 - val_accuracy: 0.7352
Epoch 4/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5516 - accuracy: 0.7312 - val_loss: 0.5439 - val_accuracy: 0.7344
Epoch 5/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7305 - val_loss: 0.5436 - val_accuracy: 0.7362
Epoch 6/100
684/684 [==============================] - 2s 2ms/step - loss: 0.5487 - accuracy: 0.7317 - val_loss: 0.5434 - val_accuracy: 0.7359
Epoch 7/100
684/684 [==============================] - 2s 3ms/step - loss: 0.5484 - accuracy: 0.7322 - val_loss: 0.5440 - val_accuracy: 0.7321

In [119]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5627 - accuracy: 0.7298 - 417ms/epoch - 2ms/step
Loss: 0.5627303123474121, Accuracy: 0.7297959327697754


In [ ]:
nn.save("AlphabetSoupCharity.h5")